In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
#non clustered dataset
df=pd.read_csv("data_regression.csv")

In [10]:
#clustered dataset (send juhi dataset)
df_c=pd.read_csv("data_clustered_4.csv")

In [11]:
df_c.head()

,Rooms,Price,Distance,Bathroom,Car,Landsize,Lattitude,Longtitude,Propertycount,month,...,CouncilArea_Yarra Ranges Shire Council,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria,clusters
0,2,1480000.0,2.5,1,1,202.0,-37.7996,144.9984,4019.0,3,...,0,0,0,1,0,0,0,0,0,0
1,2,1035000.0,2.5,1,0,156.0,-37.8079,144.9934,4019.0,4,...,0,0,0,1,0,0,0,0,0,0
2,3,1465000.0,2.5,2,0,134.0,-37.8093,144.9944,4019.0,4,...,0,0,0,1,0,0,0,0,0,0
3,3,850000.0,2.5,2,1,94.0,-37.7969,144.9969,4019.0,4,...,0,0,0,1,0,0,0,0,0,0
4,4,1600000.0,2.5,1,2,120.0,-37.8072,144.9941,4019.0,4,...,0,0,0,1,0,0,0,0,0,0


In [12]:
#get dummy for the clusters 
df_c= pd.get_dummies(df_c, columns = ['clusters'])

build rf model to get best features

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  
from scipy import stats
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
%matplotlib inline 

In [14]:
#Train Random Regressor
X = df_c.drop(['Price'], axis = 1)
y= df_c.Price
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=101)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [15]:
#Features to be tested 
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(X_train.columns, rf.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                                              score
Regionname_Southern Metropolitan            0.222613
Rooms                                       0.17122
Distance                                    0.151112
Landsize                                    0.0865337
Longtitude                                  0.077155
Lattitude                                   0.0558599
Type_u                                      0.0509883
month                                       0.0275227
Type_h                                      0.0271726
Regionname_Eastern Metropolitan             0.0267038
Bathroom                                    0.0257975
Propertycount                               0.022242
Car                                         0.0120324
CouncilArea_Stonnington City Council        0.00760552
Method_S                                    0.00633384
Method_PI                                   0.0045007
Method_VB                                   0.00423972
CouncilArea_Kingston City Cou

In [16]:
#best features based on decimal points judgement and include target variable
best_features=['Regionname_Southern Metropolitan','Rooms','Distance','Landsize','Longtitude','Lattitude','Type_u','month','Type_h',
               'Regionname_Eastern Metropolitan','Bathroom','Propertycount','Car','Price','CouncilArea_Stonnington City Council'
              ,'Method_S','Method_PI','Method_VB','CouncilArea_Kingston City Council','Method_SP','Type_t','CouncilArea_Boroondara City Council'
              ,'CouncilArea_Monash City Council','CouncilArea_Moonee Valley City Council']

GBM MODEL with new features 

In [17]:
df_m=df_c[best_features]
y= df_m['Price']
X = df_m.drop(['Price'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
#base case error without tuning 
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor

clf = GradientBoostingRegressor().fit(X_train, y_train)

In [24]:
rmse = np.sqrt(mean_squared_error(y_test,clf.predict(X_test)))
rmse

334113.86904414993

Parameter tuning for gbm

In [20]:
from sklearn.grid_search import GridSearchCV
p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01], 'n_estimators':[100,250,500,750]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(max_depth=4, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test3, scoring='explained_variance',n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.grid_scores_, tuning.best_params_, tuning.best_score_

c:\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


([mean: 0.75530, std: 0.02653, params: {'learning_rate': 0.15, 'n_estimators': 100},
  mean: 0.77451, std: 0.02677, params: {'learning_rate': 0.15, 'n_estimators': 250},
  mean: 0.78247, std: 0.02738, params: {'learning_rate': 0.15, 'n_estimators': 500},
  mean: 0.78456, std: 0.02722, params: {'learning_rate': 0.15, 'n_estimators': 750},
  mean: 0.74920, std: 0.02490, params: {'learning_rate': 0.1, 'n_estimators': 100},
  mean: 0.77157, std: 0.02555, params: {'learning_rate': 0.1, 'n_estimators': 250},
  mean: 0.78235, std: 0.02556, params: {'learning_rate': 0.1, 'n_estimators': 500},
  mean: 0.78613, std: 0.02420, params: {'learning_rate': 0.1, 'n_estimators': 750},
  mean: 0.72034, std: 0.02172, params: {'learning_rate': 0.05, 'n_estimators': 100},
  mean: 0.75850, std: 0.02399, params: {'learning_rate': 0.05, 'n_estimators': 250},
  mean: 0.77484, std: 0.02513, params: {'learning_rate': 0.05, 'n_estimators': 500},
  mean: 0.78320, std: 0.02549, params: {'learning_rate': 0.05, 'n_est

In [21]:
#input parameter from previous chunk - learning rate and n_estimators
p_test2 = {'max_depth':[2,3,4,5,6,7] }
tuning = GridSearchCV(estimator =GradientBoostingRegressor(learning_rate=0.1,n_estimators=750, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test2, scoring='explained_variance',n_jobs=4,iid=False, cv=5)
tuning.fit(X_train,y_train)
tuning.grid_scores_, tuning.best_params_, tuning.best_score_

([mean: 0.74996, std: 0.02323, params: {'max_depth': 2},
  mean: 0.77462, std: 0.02376, params: {'max_depth': 3},
  mean: 0.78613, std: 0.02420, params: {'max_depth': 4},
  mean: 0.78975, std: 0.02693, params: {'max_depth': 5},
  mean: 0.78756, std: 0.02623, params: {'max_depth': 6},
  mean: 0.78461, std: 0.02427, params: {'max_depth': 7}],
 {'max_depth': 5},
 0.7897546469995103)

In [22]:
#Evaluation on test set 
#input max depth optimal value from previous chunk which is 5 
model1 = GradientBoostingRegressor(learning_rate=0.1, n_estimators=750,max_depth=5, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
model1.fit(X_train,y_train)
rmse2 = np.sqrt(mean_squared_error(y_test,model1.predict(X_test)))
rmse2

292033.39337833639